In [ ]:
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.load import dumps, loads

In [2]:
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader('science.txt')
text = text_loader.load()

In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
docs = splitter.split_documents(text)

In [6]:
embed_model = OllamaEmbeddings(model='llama3.2')

vector = FAISS.from_documents(documents=docs,embedding=embed_model)
vector.save_local('RAGfusion')

# new_vector=FAISS.load_local("faiss_index",embeddings=embed_model,allow_dangerous_deserialization=True)

In [115]:
new_vector=FAISS.load_local("RAGfusion",embeddings=embed_model,allow_dangerous_deserialization=True).as_retriever()
question = 'Tell me about three major branches of modern science.'
# embed = embed_model.embed_query(question)
# new_vector.similarity_search(question)


In [116]:
new_vector.invoke(question)

[Document(metadata={'source': 'science.txt'}, page_content='policies that seek to influence the scientific enterprise by prioritizing the ethical and moral development of commercial products, armaments, health care, public infrastructure, and environmental protection.'),
 Document(metadata={'source': 'science.txt'}, page_content='whether the formal sciences are scientific disciplines,[8][9][10] as they do not rely on empirical evidence.[11][9] Applied sciences are disciplines that use scientific knowledge for practical purposes, such as in engineering and medicine.[12][13][14]'),
 Document(metadata={'source': 'science.txt'}, page_content='and societies;[4][5] and the formal sciences (e.g., logic, mathematics, and theoretical computer science), which study formal systems, governed by axioms and rules.[6][7] There is disagreement whether the formal sciences are scientific disciplines,[8][9][10] as they do not rely on empirical evidence.[11][9] Applied sciences are disciplines that use sc

In [64]:
prompt = ChatPromptTemplate.from_messages([
    ('system','You are a helpful assistant that generate multiple queries based on a single input query.'),
    ('user','Generate multiple search queries related to : {question} \n OUTPUT (4 queries)')
])

generate_query = prompt | ChatOllama(model='llama3.2') | StrOutputParser() | (lambda x: x.split('\n'))

In [58]:
ragfusion_chain = generate_query | new_vector.map()

In [65]:
# value =ragfusion_chain.invoke({'question':'what are modern science?'})
value = generate_query.invoke({'question':'what are modern science?'})

In [56]:
ragfusion_chain

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant that generate multiple queries based on a single input query.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Generate multiple search queries related to : {question} \n OUTPUT (4 queries)'))])
| ChatOllama(model='llama3.2')
| StrOutputParser()
| RunnableLambda(lambda x: x.split('\n'))
| RunnableEach(bound=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C29107E9C0>))
| RunnableLambda(summarize)

In [66]:
value

['Here are four different search queries related to "Modern Science":',
 '',
 '1. **"Emerging trends in scientific research 2023"**',
 '',
 'This query can help you find information on the latest advancements and breakthroughs in various fields of science, such as physics, biology, chemistry, and more.',
 '',
 '2. **"Science in modern society impact"**',
 '',
 'This query can lead you to articles and resources that explore how scientific discoveries and technologies are shaping our daily lives, from medicine and technology to environment and sustainability.',
 '',
 '3. **"Contemporary scientific theories and models"**',
 '',
 'This query can help you discover the latest scientific theories and models that explain various phenomena in the natural world, such as climate change, evolution, and particle physics.',
 '',
 '4. **"Applications of modern science in innovation"**',
 '',
 'This query can guide you to information on how scientists are using modern science to develop new technologi

In [112]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)


from langchain import hub

prompt = hub.pull("langchain-ai/rag-fusion-query-generation")


generate_queries = (
    prompt
    | ChatOllama(model='llama3.2')
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

## Chain for extracting relevant documents



In [122]:
val = generate_queries.invoke({'original_query':'about modern science'})
# generate_queries.invoke({'original_query':'modern science?'})

In [124]:
val

['Here are four search queries related to "modern science":',
 '',
 '1. **"Recent breakthroughs in renewable energy"** - This query could help you find updates on innovative technologies and advancements in the field of renewable energy.',
 '2. **"The impact of CRISPR gene editing on modern biology"** - This query could lead you to articles and research papers discussing the current state of CRISPR technology and its applications in various fields of biology.',
 '3. **"The latest findings in quantum computing and artificial intelligence"** - This query could help you find news and updates on the rapid progress being made in the field of quantum computing and its potential applications in AI.',
 '4. **"The science behind modern biotechnology and regenerative medicine"** - This query could lead you to articles and research papers discussing the current state of biotechnology and regenerative medicine, including recent advancements and potential breakthroughs.',
 '',
 'I hope these search

In [129]:

value = generate_queries.invoke({'original_query':'what are modern science?'})
# generate_queries.invoke({'original_query':'modern science?'})

In [132]:
value[1] = 'what are modern science?'

In [142]:
dic_que_res = {}
ls = []
for index in range(1,6):
    print(value[index])
    response = new_vector.get_relevant_documents(value[index])
    ls.append(response)
    dic_que_res[value[index]] = response

what are modern science?
1. **"What is the definition of modern science?"**
2. **"How has modern science evolved from traditional scientific methods?"**
3. **"What are the key principles and methodologies of modern science?"**
4. **"Compare the differences between ancient and modern scientific approaches."**


In [186]:
def reciprocal_rank_fusion(results: list[list],k=60):
    scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in scores:
                scores[doc_str] = 0
            previous_score = scores[doc_str]
            scores[doc_str] += 1 / (rank + k)
    reranked_results = [
        (loads(doc))#, score)
        for doc, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [187]:
ranked_data =reciprocal_rank_fusion(ls)
retriever = vector.as_retriever()

In [214]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

full_rag_fusion_chain = (
    {
        'context':retriever,
        'question':RunnablePassthrough()
    }
    | prompt
    | ChatOllama(model='llama3.2')
    | StrOutputParser()
)

In [207]:
full_rag_fusion_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C293E395B0>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])
| ChatOllama(model='llama3.2')

In [166]:
vector=FAISS.from_documents(embedding=embed_model,documents=ranked_data)

In [170]:
retriever = vector.as_retriever()

In [189]:
ranked_data

[Document(metadata={'source': 'science.txt'}, page_content='role in knowledge creation and it was not until the 19th century that many of the institutional and professional features of science began to take shape,[26][27] along with the changing of "natural philosophy" to "natural science".[28]'),
 Document(metadata={'source': 'science.txt'}, page_content='past that resemble modern science in some but not all features;[43][44][45] however, this label has also been criticized as denigrating,[46] or too suggestive of presentism, thinking about those activities only in relation to modern categories.[47]'),
 Document(metadata={'source': 'science.txt'}, page_content='whether the formal sciences are scientific disciplines,[8][9][10] as they do not rely on empirical evidence.[11][9] Applied sciences are disciplines that use scientific knowledge for practical purposes, such as in engineering and medicine.[12][13][14]'),
 Document(metadata={'source': 'science.txt'}, page_content='policies that 

In [212]:
full_rag_fusion_chain.invoke('what are modern science?')

'The context does not explicitly define what "modern science" is. However, it mentions that some aspects of past activities resemble modern science in some but not all features. Additionally, it notes that the institutional and professional features of science began to take shape in the 19th century, which suggests that modern science may refer to the developments and practices that emerged during this time period.'

In [176]:
full_rag_fusion_chain.input_schema()

RunnableParallel<context,question>Input(__root__=None)